In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, show
import numpy as np 
import os
from spindle_tracker.signal.fft import get_fft_downsampled, filter_signal, get_fft
import statsmodels as sm

In [4]:
#os.chdir("I:/simul_osci")
#os.chdir("/home/sophie/Documents/GIT/cytosim/cytosim-public.5042/oscillation")
#os.chdir("/home/sophie/Documents/ResultatsCytosim/oscillations/proba_asymetrique/res/metaphase1kt_100MTseuil_20cohesinStiffness_140KTMTstiffness/")
os.chdir("/media/sophie/Sophie/Resultats_simulations/20170507/result/metaphase0/")


In [5]:
def distance(x1, y1, x2, y2) : 
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

def middle_kinetochore_position (x1, y1, x2, y2) : 
    xposition = x1 + (x2-x1)/2
    yposition = y1 + (y2-y1)/2
    return (xposition, yposition)

def start_oscillation (couple1, couple2):

    allattached1 = couple1 [ couple1["attached\\attached"]>=1 ]
    allattached2 = couple2 [ couple2["attached\\attached"]>=1 ]
    minTimeAttached1 = allattached1["time"].min()
    minTimeAttached2 = allattached2["time"].min()

    start_oscillation = max(minTimeAttached1, minTimeAttached2) 

    return start_oscillation


def end_oscillation(solid):
    dist = distance(solid["x"], solid["y"], solid["x.1"], solid["y.1"])    
    
    solid = solid.assign(dist=dist.values)
    dist = solid[solid["dist"]>=2]
    
    end_oscillation = dist["time"].min()
    
    return end_oscillation

def centrage(solid):
    (x_kt, y_kt)= middle_kinetochore_position(data_solid["x.2"],data_solid["y.2"],data_solid["x.3"],data_solid["y.3"])
    (x_spb, y_spb)= middle_kinetochore_position(data_solid["x"],data_solid["y"],data_solid["x.1"],data_solid["y.1"])
    
    d = distance(x_spb, y_spb, x_kt, y_kt)
    return d

def stretch(solid):
    d=distance(data_solid["x.2"],data_solid["y.2"],data_solid["x.3"],data_solid["y.3"])
    return d
    

In [ ]:
list_dir_prometa = os.listdir()
#for i in range(90,101):
list_magn = []
list_freq = []
label = []
for MTseuil in range(90,101): 
    
    for cohesinStiff in range(10,110,10):
        list_magn[:] = []
        list_freq[:] = []
        label[:] = []
        for dir_prometa in list_dir_prometa: 
            if dir_prometa != "figure" :
                if dir_prometa != "simul":
                    dir_prometa_split = dir_prometa.split('_')
            if MTseuil.__str__() in dir_prometa_split[1] and cohesinStiff.__str__() in dir_prometa_split[2]:  
                list_dir_run = os.listdir(dir_prometa)
                label.append(dir_prometa_split[3][0:3])
                
                fig, axes = plt.subplots( nrows=2, ncols=2, figsize = (15, 10) ) 
                cpt = 0

                for dir_run in list_dir_run:

                    solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                    ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                    ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                    fin_oscillation = end_oscillation(data_solid)

                    tmp = data_solid[data_solid["time"]>debut_oscillation]
                    tmp = tmp[tmp["time"]<fin_oscillation]
                    x1 = tmp["x.2"]
                    y1 = tmp["y.2"]
                    x2 = tmp["x.3"]
                    y2 = tmp["y.3"]

                    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)

                    #print(type(np.array(kt_x_pos.tolist())))
                    #print (np.array(kt_x_pos.tolist()))
                    #print (kt_x_pos.tolist())
                    #if np.array(kt_x_pos.tolist()).any() :
                    #    (_list_magn, _list_freq) = get_fft((np.array(kt_x_pos.tolist())), 0.05)
                    #    list_magn.append(_list_magn)
                    #    list_freq.append(_list_freq)
                    (kt_x,kt_y)=middle_kinetochore_position(data_solid["x.2"],data_solid["y.2"],data_solid["x.3"],data_solid["y.3"])
                    
                    p1=axes[cpt,0].plot(data_solid["time"], data_solid["x"], color="black")
                    p2=axes[cpt,0].plot(data_solid["time"], data_solid["x.1"], color="black")
                    p3=axes[cpt,0].plot(data_solid["time"],kt_x, color="blue")
                    p4=axes[cpt,0].plot(data_solid["time"], data_solid["x.2"], color="yellow")
                    p5=axes[cpt,0].plot(data_solid["time"], data_solid["x.3"], color="yellow")
                    axes[cpt,0].axvline(x=debut_oscillation, linewidth=1, color='k')
                    axes[cpt,0].axvline(x=fin_oscillation, linewidth=1, color='k')
                    
                    
                    if kt_x_pos.any():
                        (x,time)=filter_signal(kt_x_pos,1,0.01)


                        # freq max = 1
                        (mag, freq) = get_fft(x, 0.05)
                        
                        
                        axes[cpt,1].plot(freq, mag)
                        axes[cpt,1].set_xlabel('Frequence (Hz)')
                        axes[cpt,1].set_ylabel('Amplitude')
                        axes[cpt,1].set_xscale('log')
                        axes[cpt,1].grid(True, which="both")
                        #plt.show()
                        cpt = cpt+1
                        

                        solid.close()
                        ktmt1.close()
                        ktmt2.close()
                        
                fig.savefig("../figure/"+str(MTseuil)+"MTseuil"+str(cohesinStiff)+"cohesinStiff"+str(dir_prometa_split[3][0:3])+".png")
                plt.close(fig)
            #else:
                #print ("pas trouve")
        
        
        #fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 10))

        #axes[0].violinplot(list_magn, showmeans=True, showmedians=True)
        #axes[0].set_title('magnitude')

        #pos   = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
        #label = ['100','110','120','130','140','150', '160', '170', '180', '190', '200']
        #axes[0].set_xticks(pos)
        #axes[0].set_xticklabels(label)
        #axes[0].set_xlabel('Stiffness', fontsize=15)
        #axes[0].set_ylabel('Magnitude', fontsize=15)


        #axes[1].violinplot(list_freq, showmeans=True, showmedians=True)
        #axes[1].set_title('frequence')
        #axes[1].set_xticks(pos)
        #axes[1].set_xticklabels(label)
        #axes[1].set_xlabel('Stiffness', fontsize=15)
        #axes[1].set_ylabel('Magnitude', fontsize=15)

        #fig.savefig("../figure/"+str(MTseuil)+"MTseuil"+str(cohesinStiff)+"cohesinStiff"+".png")
        #plt.close(fig)
        
        list_magn[:] = []
        list_freq[:] = []


In [ ]:
plt.plot(time,kt_x_pos)
plt.grid()
plt.show()
# valeur publi
#(x,time)=filter_signal(kt_x_pos,1,0.005)
(x,time)=filter_signal(kt_x_pos,1,0.01)
print(kt_x_pos.size)
print(x.size)
print(time.size)
plt.plot( time,x)
plt.grid()
plt.show()

In [ ]:
# freq max = 1
#(mag, freq)=get_fft_downsampled(x, 0.2,0.5)
#(mag, freq)=get_fft_downsampled(x, 0.05,5)

(mag, freq) = get_fft(x, 0.05)

plt.plot(freq, mag,'.', linestyle=':', marker='.')
plt.xlabel('Frequence (Hz)')
plt.ylabel('Amplitude')
plt.axis([0,1,0,0.5])
plt.grid()
plt.show()
plt.plot(freq, mag, linestyle=':', marker='.')
plt.xlabel('Frequence (Hz)')
plt.ylabel('Amplitude')
plt.xscale('log')
plt.grid(True, which="both")
plt.show()

# Figure centrage -> Normalised distance (nd)

In [ ]:
# TEST
solid  = open("metaphase1kt_100MTseuil_100cohesinStiffness_200KTMTstiffness/run2/solid.txt")
ktmt1  = open("metaphase1kt_100MTseuil_100cohesinStiffness_200KTMTstiffness/run2/couple:KTMT.txt")
ktmt2  = open("metaphase1kt_100MTseuil_100cohesinStiffness_200KTMTstiffness/run2/couple:KTMT2.txt")

data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
fin_oscillation = end_oscillation(data_solid)

tmp = data_solid[data_solid["time"]>debut_oscillation]
tmp = tmp[tmp["time"]<fin_oscillation]
x1 = tmp["x.2"]
y1 = tmp["y.2"]
x2 = tmp["x.3"]
y2 = tmp["y.3"]



(kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)
d_centre=centrage(solid)
plt.hist(d_centre, 25)
plt.show()

In [84]:
# boucle 
list_dir_prometa = os.listdir()
#for i in range(90,101):
pos = []
label = []

for MTseuil in range(90,101): 
    first=1
    i=1
    #d_centre_tot = pd.DataFrame
    d_centre_tot= list()
    for cohesinStiff in range(10,110,10):
        
        label.append(cohesinStiff)
        pos.append(i)
        i=i+1
       
        d_centre = []
        for dir_prometa in list_dir_prometa: 
             
            
            if dir_prometa != "figure" :
                if dir_prometa != "simul":
                    dir_prometa_split = dir_prometa.split('_')
                    
            if MTseuil.__str__() in dir_prometa_split[1] and cohesinStiff.__str__() in dir_prometa_split[2]:  
                list_dir_run = os.listdir(dir_prometa)


                for dir_run in list_dir_run:

                    solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                    ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                    ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                    fin_oscillation = end_oscillation(data_solid)

                    tmp = data_solid[data_solid["time"]>debut_oscillation]
                    tmp = tmp[tmp["time"]<fin_oscillation]
                    x1 = tmp["x.2"]
                    y1 = tmp["y.2"]
                    x2 = tmp["x.3"]
                    y2 = tmp["y.3"]
                    
                    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
                    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)
                    
                    
                    d_centre.extend(distance(spb_x_pos, spb_y_pos, kt_x_pos, kt_y_pos))
                        

                    solid.close()
                    ktmt1.close()
                    ktmt2.close()
        #if first==1:
        #    d={cohesinStiff:d_centre}
            #d_centre_tot = pd.DataFrame(data=d)
        #    first = 0
        
        #d={cohesinStiff:d_centre}
        #tmp = pd.DataFrame(data=d)
        #d_centre_tot.append(d)
        #d_centre_tot= pd.merge(d_centre_tot, tmp, how='right')
        d_centre_tot.append(list(d_centre))
    
    
    fig,ax = plt.subplots(figsize=(10,5))
    ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
    ax.set_xticks(pos)
    ax.set_xticklabels(label)
    ax.set_ylim([0,0.5])
    ax.set_xlabel('Stiffness Cohesin', fontsize=15)
    ax.set_ylabel('Normalised Distance (nd)', fontsize=15)
    
    
    plt.savefig("../Figures/figure_centrage_MT/"+str(MTseuil)+"MTseuil.png")
    plt.close(fig)

/home/sophie/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [85]:
# boucle 
list_dir_prometa = os.listdir()
#for i in range(90,101):
pos = []
label = []

for cohesinStiff in range(10,110,10):
 
    first=1
    i=1
    #d_centre_tot = pd.DataFrame
    d_centre_tot= list()
    for MTseuil in range(90,101):
        
        label.append(MTseuil)
        pos.append(i)
        i=i+1
       
        d_centre = []
        for dir_prometa in list_dir_prometa: 
             
            
            if dir_prometa != "figure" :
                if dir_prometa != "simul":
                    dir_prometa_split = dir_prometa.split('_')
                    
            if MTseuil.__str__() in dir_prometa_split[1] and cohesinStiff.__str__() in dir_prometa_split[2]:  
                list_dir_run = os.listdir(dir_prometa)


                for dir_run in list_dir_run:

                    solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                    ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                    ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                    fin_oscillation = end_oscillation(data_solid)

                    tmp = data_solid[data_solid["time"]>debut_oscillation]
                    tmp = tmp[tmp["time"]<fin_oscillation]
                    x1 = tmp["x.2"]
                    y1 = tmp["y.2"]
                    x2 = tmp["x.3"]
                    y2 = tmp["y.3"]

                    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
                    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)
                    
                    
                    d_centre.extend(distance(spb_x_pos, spb_y_pos, kt_x_pos, kt_y_pos))
                        

                    solid.close()
                    ktmt1.close()
                    ktmt2.close()
        #if first==1:
        #    d={cohesinStiff:d_centre}
            #d_centre_tot = pd.DataFrame(data=d)
        #    first = 0
        
        #d={cohesinStiff:d_centre}
        #tmp = pd.DataFrame(data=d)
        #d_centre_tot.append(d)
        #d_centre_tot= pd.merge(d_centre_tot, tmp, how='right')
        d_centre_tot.append(list(d_centre))
    
    
    fig,ax = plt.subplots(figsize=(10,5))
    ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
    ax.set_xticks(pos)
    ax.set_xticklabels(label)
    ax.set_ylim([0,0.5])
    ax.set_xlabel('Seuil MT', fontsize=15)
    ax.set_ylabel('Normalised Distance (nd)', fontsize=15)
    
    
    plt.savefig("../Figures/figure_centrage_Cohesin/"+str(cohesinStiff)+"cohesinStiffness.png")
    plt.close(fig)



/home/sophie/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [15]:
# boucle 
list_dir_prometa = os.listdir()
#for i in range(90,101):
pos = []
label = []
i=1
d_centre_tot= list()
for MTseuil in range(100,101,1): 
    first=1
    
    #d_centre_tot = pd.DataFrame
    
    for KTMT in range(20,120,20):
        
        label.append(str(MTseuil)+"-"+str(KTMT))
        pos.append(i)
        i=i+1
       
        d_centre = []
        for dir_prometa in list_dir_prometa: 
             
            
            if dir_prometa != "figure" :
                if dir_prometa != "simul":
                    dir_prometa_split = dir_prometa.split('_')
                    
            if MTseuil.__str__() in dir_prometa_split[1] and KTMT.__str__() in dir_prometa_split[3]:  
                list_dir_run = os.listdir(dir_prometa)


                for dir_run in list_dir_run:

                    solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                    ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                    ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                    fin_oscillation = end_oscillation(data_solid)

                    tmp = data_solid[data_solid["time"]>debut_oscillation]
                    tmp = tmp[tmp["time"]<fin_oscillation]
                    x1 = tmp["x.2"]
                    y1 = tmp["y.2"]
                    x2 = tmp["x.3"]
                    y2 = tmp["y.3"]
                    
                    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
                    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)
                    
                    
                    d_centre.extend(distance(spb_x_pos, spb_y_pos, kt_x_pos, kt_y_pos))
                        

                    solid.close()
                    ktmt1.close()
                    ktmt2.close()
        #if first==1:
        #    d={cohesinStiff:d_centre}
            #d_centre_tot = pd.DataFrame(data=d)
        #    first = 0
        
        #d={cohesinStiff:d_centre}
        #tmp = pd.DataFrame(data=d)
        #d_centre_tot.append(d)
        #d_centre_tot= pd.merge(d_centre_tot, tmp, how='right')
        d_centre_tot.append(list(d_centre))
    
    
fig,ax = plt.subplots(figsize=(10,5))
ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
ax.set_xticks(pos)
ax.set_xticklabels(label, rotation='vertical')
ax.set_ylim([0,0.8])
ax.set_xlabel('Stiffness KTMT', fontsize=15)
ax.set_ylabel('Normalised Distance (nd)', fontsize=15)


plt.savefig("../Figures/centrage.png")
plt.close(fig)

In [130]:
# boucle 
list_dir_prometa = os.listdir()
#for i in range(90,101):
pos = []
label = []

for KTMT in range(100,210,10):
 
    first=1
    i=1
    #d_centre_tot = pd.DataFrame
    d_centre_tot= list()
    for MTseuil in range(90,101):
        
        label.append(MTseuil)
        pos.append(i)
        i=i+1
       
        d_centre = []
        for dir_prometa in list_dir_prometa: 
             
            
            if dir_prometa != "figure" :
                if dir_prometa != "simul":
                    dir_prometa_split = dir_prometa.split('_')
                    
            if MTseuil.__str__() in dir_prometa_split[1] and KTMT.__str__() in dir_prometa_split[3]:  
                list_dir_run = os.listdir(dir_prometa)


                for dir_run in list_dir_run:

                    solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                    ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                    ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                    fin_oscillation = end_oscillation(data_solid)

                    tmp = data_solid[data_solid["time"]>debut_oscillation]
                    tmp = tmp[tmp["time"]<fin_oscillation]
                    x1 = tmp["x.2"]
                    y1 = tmp["y.2"]
                    x2 = tmp["x.3"]
                    y2 = tmp["y.3"]

                    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
                    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)
                    
                    
                    d_centre.extend(distance(spb_x_pos, spb_y_pos, kt_x_pos, kt_y_pos))
                        

                    solid.close()
                    ktmt1.close()
                    ktmt2.close()
        #if first==1:
        #    d={cohesinStiff:d_centre}
            #d_centre_tot = pd.DataFrame(data=d)
        #    first = 0
        
        #d={cohesinStiff:d_centre}
        #tmp = pd.DataFrame(data=d)
        #d_centre_tot.append(d)
        #d_centre_tot= pd.merge(d_centre_tot, tmp, how='right')
        d_centre_tot.append(list(d_centre))
    
    
    fig,ax = plt.subplots(figsize=(10,5))
    ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
    ax.set_xticks(pos)
    ax.set_xticklabels(label)
    ax.set_ylim([0,0.5])
    ax.set_xlabel('Seuil MT', fontsize=15)
    ax.set_ylabel('Normalised Distance (nd)', fontsize=15)
    
    
    plt.savefig("../Figures/MT_KTMT/figure_centrage_KTMT/"+str(KTMT)+"KTMTStiffness.png")
    plt.close(fig)



In [10]:
# boucle  juste seuil MT
list_dir_prometa = os.listdir()
#for i in range(90,101):
pos = []
label = []
i=1
d_centre_tot= list()
for MTseuil in range(97,101,2): 
    first=1
    
    #d_centre_tot = pd.DataFrame

    #for cohesinStiff in range(10,110,10):
        
    label.append(MTseuil)
    pos.append(i)
    i=i+1

    d_centre = []
    for dir_prometa in list_dir_prometa: 


        if dir_prometa != "figure" :
            if dir_prometa != "simul":
                dir_prometa_split = dir_prometa.split('_')

        if MTseuil.__str__() in dir_prometa_split[1]:  
            list_dir_run = os.listdir(dir_prometa)


            for dir_run in list_dir_run:

                solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                fin_oscillation = end_oscillation(data_solid)

                tmp = data_solid[data_solid["time"]>debut_oscillation]
                tmp = tmp[tmp["time"]<fin_oscillation]
                x1 = tmp["x.2"]
                y1 = tmp["y.2"]
                x2 = tmp["x.3"]
                y2 = tmp["y.3"]

                (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
                (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)


                if kt_x_pos.any():
                    (x,time)=filter_signal(kt_x_pos,1,0.01)

                    # freq max = 1
                    (mag, freq) = get_fft(x, 0.05)
                    #amplitude=list()
                    #amplitude = list(mag)
                    #for elt in mag:
                    #    if elt > 0.01:
                    #        amplitude.append(elt)
                    amplitude = np.amax(mag)
                    d_centre.append(amplitude)


                solid.close()
                ktmt1.close()
                ktmt2.close()

    d_centre_tot.append(list(d_centre))
    
    
fig,ax = plt.subplots(figsize=(10,5))
ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
ax.set_xticks(pos)
ax.set_xticklabels(label)
ax.set_ylim([0,1])
ax.set_xlabel('Seuil MT', fontsize=15)
ax.set_ylabel('Amplitude', fontsize=15)


plt.savefig("../Figures/amplitude.png")
plt.close(fig)

In [7]:
os.chdir ("/media/sophie/Sophie/Resultats_simulations/20170507/result/metaphase0/")
pos = []
label = []
i=1
d_centre_tot= list()

first=1


pos.append(i)
i=i+1

d_centre = []

list_dir_run = os.listdir()


for dir_run in list_dir_run[:-1]:

    solid  = open(dir_run+"/solid.txt")
    ktmt1  = open(dir_run+"/couple:KTMT.txt")
    ktmt2  = open(dir_run+"/couple:KTMT2.txt")

    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

    """debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
    fin_oscillation = end_oscillation(data_solid)

    tmp = data_solid[data_solid["time"]>debut_oscillation]
    tmp = tmp[tmp["time"]<fin_oscillation]
    x1 = tmp["x.2"]
    y1 = tmp["y.2"]
    x2 = tmp["x.3"]
    y2 = tmp["y.3"]

    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)


    if kt_x_pos.any():
        (x,time)=filter_signal(kt_x_pos,1,0.01)

        # freq max = 1
        (mag, freq) = get_fft(x, 0.05)
        #amplitude=list()
        #amplitude = list(mag)
        #for elt in mag:
        #    if elt > 0.01:
        #        amplitude.append(elt)
        amplitude = np.amax(mag)
        d_centre.append(amplitude)


    solid.close()
    ktmt1.close()
    ktmt2.close()"""


    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
    fin_oscillation = end_oscillation(data_solid)

    tmp = data_solid[data_solid["time"]>debut_oscillation]
    tmp = tmp[tmp["time"]<fin_oscillation]
    x1 = tmp["x.2"]
    y1 = tmp["y.2"]
    x2 = tmp["x.3"]
    y2 = tmp["y.3"]

    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)


    d_centre.extend(distance(spb_x_pos, spb_y_pos, kt_x_pos, kt_y_pos)/distance(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"]))


    solid.close()
    ktmt1.close()
    ktmt2.close()

d_centre_tot.append(list(d_centre))

    
fig,ax = plt.subplots(figsize=(3,10))
ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
ax.set_xticks(pos)
ax.set_xticklabels(label)
ax.set_ylim([0,0.5])
ax.set_xlabel('', fontsize=15)
ax.set_ylabel('Normalised Distance (nd)', fontsize=15)


plt.savefig("../ND.png")
plt.close(fig)

# STRETCH

In [ ]:
d_stretch= stretch(solid)
plt.hist(d_stretch,25)
plt.show()

In [87]:
# boucle 
list_dir_prometa = os.listdir()
#for i in range(90,101):
pos = []
label = []

for MTseuil in range(90,101): 
    first=1
    i=1
    #d_centre_tot = pd.DataFrame
    d_centre_tot= list()
    for cohesinStiff in range(10,110,10):
        
        label.append(cohesinStiff)
        pos.append(i)
        i=i+1
       
        d_centre = []
        for dir_prometa in list_dir_prometa: 
             
            
            if dir_prometa != "figure" :
                if dir_prometa != "simul":
                    dir_prometa_split = dir_prometa.split('_')
                    
            if MTseuil.__str__() in dir_prometa_split[1] and cohesinStiff.__str__() in dir_prometa_split[2]:  
                list_dir_run = os.listdir(dir_prometa)


                for dir_run in list_dir_run:

                    solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                    ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                    ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                    fin_oscillation = end_oscillation(data_solid)

                    tmp = data_solid[data_solid["time"]>debut_oscillation]
                    tmp = tmp[tmp["time"]<fin_oscillation]
                    x1 = tmp["x.2"]
                    y1 = tmp["y.2"]
                    x2 = tmp["x.3"]
                    y2 = tmp["y.3"]
                    
                    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
                    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)
                    
                    #def stretch(solid):
                    #d=distance(data_solid["x.2"],data_solid["y.2"],data_solid["x.3"],data_solid["y.3"])
                    #return d
                    
                    d_centre.extend(distance(x1,y1,x2,y2))
                        

                    solid.close()
                    ktmt1.close()
                    ktmt2.close()
        #if first==1:
        #    d={cohesinStiff:d_centre}
            #d_centre_tot = pd.DataFrame(data=d)
        #    first = 0
        
        #d={cohesinStiff:d_centre}
        #tmp = pd.DataFrame(data=d)
        #d_centre_tot.append(d)
        #d_centre_tot= pd.merge(d_centre_tot, tmp, how='right')
        d_centre_tot.append(list(d_centre))
    
    
    fig,ax = plt.subplots(figsize=(10,5))
    ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
    ax.set_xticks(pos)
    ax.set_xticklabels(label)
    ax.set_ylim([0,1])
    ax.set_xlabel('Stiffness Cohesin', fontsize=15)
    ax.set_ylabel('Stretch(µm)', fontsize=15)
    
    
    plt.savefig("../Figures/figure_stretch_MT/"+str(MTseuil)+"MTseuil.png")
    plt.close(fig)

/home/sophie/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [70]:
#os.chdir("/home/sophie/Documents/ResultatsCytosim/oscillations/simul_osci")
# boucle 
list_dir_prometa = os.listdir()
#for i in range(90,101):
pos = []
label = []
first=1
i=1
d_centre_tot= list()
for cohesinStiff in range(10,110,10):
   
    #for MTseuil in range(90,101):
        
    label.append(cohesinStiff)
    pos.append(i)
    i=i+1

    d_centre = []
    for dir_prometa in list_dir_prometa: 


        if dir_prometa != "figure" :
            if dir_prometa != "simul":
                dir_prometa_split = dir_prometa.split('_')

        if cohesinStiff.__str__() in dir_prometa_split[2]:  
            list_dir_run = os.listdir(dir_prometa)


            for dir_run in list_dir_run:

                solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                fin_oscillation = end_oscillation(data_solid)

                tmp = data_solid[data_solid["time"]>debut_oscillation]
                tmp = tmp[tmp["time"]<fin_oscillation]
                x1 = tmp["x.2"]
                y1 = tmp["y.2"]
                x2 = tmp["x.3"]
                y2 = tmp["y.3"]

                (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
                (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)


                d_centre.extend(distance(x1,y1,x2,y2))


                solid.close()
                ktmt1.close()
                ktmt2.close()
    #if first==1:
    #    d={cohesinStiff:d_centre}
        #d_centre_tot = pd.DataFrame(data=d)
    #    first = 0

    #d={cohesinStiff:d_centre}
    #tmp = pd.DataFrame(data=d)
    #d_centre_tot.append(d)
    #d_centre_tot= pd.merge(d_centre_tot, tmp, how='right')
    d_centre_tot.append(list(d_centre))


fig,ax = plt.subplots(figsize=(10,5))
ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
ax.set_xticks(pos)
ax.set_xticklabels(label)
ax.set_ylim([0,1])
ax.set_xlabel('Stiffness Cohesin (pN/µm)', fontsize=15)
ax.set_ylabel('Stretch(µm)', fontsize=15)


plt.savefig("../Figures/stretch.png")
plt.close(fig)



FileNotFoundError: [Errno 2] No such file or directory: '/home/sophie/Documents/ResultatsCytosim/oscillations/simul_osci'

In [16]:
# boucle 
list_dir_prometa = os.listdir()
#for i in range(90,101):
pos = []
label = []
i=1
d_centre_tot= list()
for MTseuil in range(100,101,1): 
    first=1
    
    #d_centre_tot = pd.DataFrame
    
    for KTMT in range(20,120,20):
        
        label.append(str(MTseuil)+"-"+str(KTMT))
        pos.append(i)
        i=i+1
       
        d_centre = []
        for dir_prometa in list_dir_prometa: 
             
            
            if dir_prometa != "figure" :
                if dir_prometa != "simul":
                    dir_prometa_split = dir_prometa.split('_')
                    
            if MTseuil.__str__() in dir_prometa_split[1] and KTMT.__str__() in dir_prometa_split[3]:  
                list_dir_run = os.listdir(dir_prometa)


                for dir_run in list_dir_run:

                    solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                    ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                    ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                    fin_oscillation = end_oscillation(data_solid)

                    tmp = data_solid[data_solid["time"]>debut_oscillation]
                    tmp = tmp[tmp["time"]<fin_oscillation]
                    x1 = tmp["x.2"]
                    y1 = tmp["y.2"]
                    x2 = tmp["x.3"]
                    y2 = tmp["y.3"]

                    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
                    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)


                    d_centre.extend(distance(x1,y1,x2,y2))
                        

                    solid.close()
                    ktmt1.close()
                    ktmt2.close()
        #if first==1:
        #    d={cohesinStiff:d_centre}
            #d_centre_tot = pd.DataFrame(data=d)
        #    first = 0
        
        #d={cohesinStiff:d_centre}
        #tmp = pd.DataFrame(data=d)
        #d_centre_tot.append(d)
        #d_centre_tot= pd.merge(d_centre_tot, tmp, how='right')
        d_centre_tot.append(list(d_centre))
    
    
fig,ax = plt.subplots(figsize=(10,5))
ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
ax.set_xticks(pos)
ax.set_xticklabels(label, rotation='vertical')
ax.set_ylim([0,0.8])
ax.set_xlabel('Stiffness KTMT', fontsize=15)
ax.set_ylabel('Stretch(µm)', fontsize=15)

plt.savefig("../Figures/stretch.png")
plt.close(fig)

# Amplitude 

In [118]:
# boucle 
list_dir_prometa = os.listdir()
#for i in range(90,101):
pos = []
label = []

for MTseuil in range(90,101): 
    first=1
    i=1
    #d_centre_tot = pd.DataFrame
    d_centre_tot= list()
    for cohesinStiff in range(10,110,10):
        
        label.append(cohesinStiff)
        pos.append(i)
        i=i+1
       
        d_centre = []
        for dir_prometa in list_dir_prometa: 
             
            
            if dir_prometa != "figure" :
                if dir_prometa != "simul":
                    dir_prometa_split = dir_prometa.split('_')
                    
            if MTseuil.__str__() in dir_prometa_split[1] and cohesinStiff.__str__() in dir_prometa_split[2]:  
                list_dir_run = os.listdir(dir_prometa)


                for dir_run in list_dir_run:

                    solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                    ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                    ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                    fin_oscillation = end_oscillation(data_solid)

                    tmp = data_solid[data_solid["time"]>debut_oscillation]
                    tmp = tmp[tmp["time"]<fin_oscillation]
                    x1 = tmp["x.2"]
                    y1 = tmp["y.2"]
                    x2 = tmp["x.3"]
                    y2 = tmp["y.3"]
                    
                    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
                    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)
                    
                    if kt_x_pos.any():
                        (x,time)=filter_signal(kt_x_pos,1,0.01)

                        # freq max = 1
                        (mag, freq) = get_fft(x, 0.05)
                        #amplitude=list()
                        #amplitude = list(mag)
                        #for elt in mag:
                        #    if elt > 0.01:
                        #        amplitude.append(elt)
                        amplitude = np.amax(mag)
                        d_centre.append(amplitude)
                        

                    solid.close()
                    ktmt1.close()
                    ktmt2.close()
        #if first==1:
        #    d={cohesinStiff:d_centre}
            #d_centre_tot = pd.DataFrame(data=d)
        #    first = 0
        
        #d={cohesinStiff:d_centre}
        #tmp = pd.DataFrame(data=d)
        #d_centre_tot.append(d)
        #d_centre_tot= pd.merge(d_centre_tot, tmp, how='right')
        d_centre_tot.append(list(d_centre))
    
    
    fig,ax = plt.subplots(figsize=(10,5))
    ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
    ax.set_xticks(pos)
    ax.set_xticklabels(label)
    ax.set_ylim([0,1])
    ax.set_xlabel('Stiffness Cohesin', fontsize=15)
    ax.set_ylabel('Amplitude(µm)', fontsize=15)
    
    
    plt.savefig("../Figures/figure_amplitude_MT/"+str(MTseuil)+"MTseuil.png")
    plt.close(fig)

/home/sophie/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [117]:
# boucle 
list_dir_prometa = os.listdir()
#for i in range(90,101):
pos = []
label = []

for cohesinStiff in range(10,110,10):
 
    first=1
    i=1
    #d_centre_tot = pd.DataFrame
    d_centre_tot= list()
    for MTseuil in range(90,101):
        
        label.append(MTseuil)
        pos.append(i)
        i=i+1
       
        d_centre = []
        for dir_prometa in list_dir_prometa: 
             
            
            if dir_prometa != "figure" :
                if dir_prometa != "simul":
                    dir_prometa_split = dir_prometa.split('_')
                    
            if MTseuil.__str__() in dir_prometa_split[1] and cohesinStiff.__str__() in dir_prometa_split[2]:  
                list_dir_run = os.listdir(dir_prometa)


                for dir_run in list_dir_run:

                    solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                    ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                    ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                    fin_oscillation = end_oscillation(data_solid)

                    tmp = data_solid[data_solid["time"]>debut_oscillation]
                    tmp = tmp[tmp["time"]<fin_oscillation]
                    x1 = tmp["x.2"]
                    y1 = tmp["y.2"]
                    x2 = tmp["x.3"]
                    y2 = tmp["y.3"]

                    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
                    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)
                    
                    
                    if kt_x_pos.any():
                        (x,time)=filter_signal(kt_x_pos,1,0.01)

                        # freq max = 1
                        (mag, freq) = get_fft(x, 0.05)
                        #amplitude=list()
                        #amplitude = list(mag)
                        #for elt in mag:
                        #    if elt > 0.01:
                        #        amplitude.append(elt)
                        amplitude = np.amax(mag)
                        d_centre.append(amplitude)
                        

                    solid.close()
                    ktmt1.close()
                    ktmt2.close()
        #if first==1:
        #    d={cohesinStiff:d_centre}
            #d_centre_tot = pd.DataFrame(data=d)
        #    first = 0
        
        #d={cohesinStiff:d_centre}
        #tmp = pd.DataFrame(data=d)
        #d_centre_tot.append(d)
        #d_centre_tot= pd.merge(d_centre_tot, tmp, how='right')
        d_centre_tot.append(list(d_centre))
    
    
    fig,ax = plt.subplots(figsize=(10,5))
    ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
    ax.set_xticks(pos)
    ax.set_xticklabels(label)
    ax.set_ylim([0,1])
    ax.set_xlabel('Seuil MT', fontsize=15)
    ax.set_ylabel('Amplitude(µm)', fontsize=15)
    
    
    plt.savefig("../Figures/figure_amplitude_Cohesin/"+str(cohesinStiff)+"cohesinStiffness.png")
    plt.close(fig)



/home/sophie/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [12]:
# boucle 
list_dir_prometa = os.listdir()
#for i in range(90,101):
pos = []
label = []

for MTseuil in range(97,101): 
    first=1
    i=1
    #d_centre_tot = pd.DataFrame
    d_centre_tot= list()
    for KTMT in range(20,100,20):
        
        label.append(KTMT)
        pos.append(i)
        i=i+1
       
        d_centre = []
        for dir_prometa in list_dir_prometa: 
             
            
            if dir_prometa != "figure" :
                if dir_prometa != "simul":
                    dir_prometa_split = dir_prometa.split('_')
                    
            if MTseuil.__str__() in dir_prometa_split[1] and KTMT.__str__() in dir_prometa_split[3]:  
                list_dir_run = os.listdir(dir_prometa)


                for dir_run in list_dir_run:

                    solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                    ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                    ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                    fin_oscillation = end_oscillation(data_solid)

                    tmp = data_solid[data_solid["time"]>debut_oscillation]
                    tmp = tmp[tmp["time"]<fin_oscillation]
                    x1 = tmp["x.2"]
                    y1 = tmp["y.2"]
                    x2 = tmp["x.3"]
                    y2 = tmp["y.3"]
                    
                    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
                    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)
                    
                    if kt_x_pos.any():
                        (x,time)=filter_signal(kt_x_pos,1,0.01)

                        # freq max = 1
                        (mag, freq) = get_fft(x, 0.05)
                        #amplitude=list()
                        #amplitude = list(mag)
                        #for elt in mag:
                        #    if elt > 0.01:
                        #        amplitude.append(elt)
                        amplitude = np.amax(mag)
                        d_centre.append(amplitude)
                        

                    solid.close()
                    ktmt1.close()
                    ktmt2.close()
        #if first==1:
        #    d={cohesinStiff:d_centre}
            #d_centre_tot = pd.DataFrame(data=d)
        #    first = 0
        
        #d={cohesinStiff:d_centre}
        #tmp = pd.DataFrame(data=d)
        #d_centre_tot.append(d)
        #d_centre_tot= pd.merge(d_centre_tot, tmp, how='right')
        d_centre_tot.append(list(d_centre))
    
    
    fig,ax = plt.subplots(figsize=(10,5))
    ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
    ax.set_xticks(pos)
    ax.set_xticklabels(label)
    ax.set_ylim([0,1])
    ax.set_xlabel('Stiffness KTMT', fontsize=15)
    ax.set_ylabel('Amplitude(µm)', fontsize=15)
    
    
    plt.savefig("../exploration2/Figures/figure_amplitude/"+str(MTseuil)+"MTseuil.png")
    plt.close(fig)

ValueError: zero-size array to reduction operation minimum which has no identity

In [9]:
# boucle 
list_dir_run = os.listdir()
#for i in range(90,101):
pos = []
label = []
d_centre_tot= list()
i=1
 
first=1



pos.append(i)
i=i+1

d_centre = []



for dir_run in list_dir_run:

    solid  = open(dir_run+"/solid.txt")
    ktmt1  = open(dir_run+"/couple:KTMT.txt")
    ktmt2  = open(dir_run+"/couple:KTMT2.txt")

    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
    fin_oscillation = end_oscillation(data_solid)

    tmp = data_solid[data_solid["time"]>(debut_oscillation+20)]
    tmp = tmp[tmp["time"]<fin_oscillation-20]
    x1 = tmp["x.2"]
    y1 = tmp["y.2"]
    x2 = tmp["x.3"]
    y2 = tmp["y.3"]

    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)

    if kt_x_pos.any():
        (x,time)=filter_signal(kt_x_pos,1,0.01)

        # freq max = 1
        (mag, freq) = get_fft(x, 0.05)
        #amplitude=list()
        #amplitude = list(mag)
        #for elt in mag:
        #    if elt > 0.01:
        #        amplitude.append(elt)
        amplitude = np.amax(mag)
        d_centre.append(amplitude)


    solid.close()
    ktmt1.close()
    ktmt2.close()
#if first==1:
#    d={cohesinStiff:d_centre}
    #d_centre_tot = pd.DataFrame(data=d)
#    first = 0

#d={cohesinStiff:d_centre}
#tmp = pd.DataFrame(data=d)
#d_centre_tot.append(d)
#d_centre_tot= pd.merge(d_centre_tot, tmp, how='right')
d_centre_tot.append(list(d_centre))
    
    

In [10]:
d_centre_tot

[[0.099809487109321499,
  0.22762054794520561,
  0.092455020561481402,
  0.14499865454902666,
  0.10665317549001685,
  0.13454789272030665,
  0.2260903141361256,
  0.17764328358208953,
  0.11400056974949309,
  0.085532635908110929,
  0.083774918978571519,
  0.22841569767441858,
  0.089450251417173124,
  0.19763733333333319,
  0.13236061381074171,
  0.11168760167994161,
  0.10382820461988666,
  0.11212292362246225,
  0.75694354838709688,
  0.15867719780219783,
  0.086332521661181952,
  0.26282010582010584,
  0.37492173913043497,
  0.44556470588235297,
  0.087072100313479631,
  0.28468518518518521,
  0.33760427807486643,
  0.16972322254084046,
  0.20326912181303122,
  0.093054276315789539,
  0.32849348958333341,
  0.17513190954773866,
  0.11155899395474142,
  0.064311704266487663,
  0.093999492509554111,
  0.16225000000000003,
  0.087074656307985526,
  0.19170557491289197,
  0.12402270751632453,
  0.1498487084870849,
  0.15637632978723415,
  0.35648133333333332,
  0.11870759088077681,
  

In [125]:
# boucle 
list_dir_prometa = os.listdir()
#for i in range(90,101):
pos = []
label = []

for KTMT in range(100,210,10):
 
    first=1
    i=1
    #d_centre_tot = pd.DataFrame
    d_centre_tot= list()
    for MTseuil in range(90,101):
        
        label.append(MTseuil)
        pos.append(i)
        i=i+1
       
        d_centre = []
        for dir_prometa in list_dir_prometa: 
             
            
            if dir_prometa != "figure" :
                if dir_prometa != "simul":
                    dir_prometa_split = dir_prometa.split('_')
                    
            if MTseuil.__str__() in dir_prometa_split[1] and KTMT.__str__() in dir_prometa_split[3]:  
                list_dir_run = os.listdir(dir_prometa)


                for dir_run in list_dir_run:

                    solid  = open(dir_prometa+"/"+dir_run+"/solid.txt")
                    ktmt1  = open(dir_prometa+"/"+dir_run+"/couple:KTMT.txt")
                    ktmt2  = open(dir_prometa+"/"+dir_run+"/couple:KTMT2.txt")

                    data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
                    data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

                    debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
                    fin_oscillation = end_oscillation(data_solid)

                    tmp = data_solid[data_solid["time"]>debut_oscillation]
                    tmp = tmp[tmp["time"]<fin_oscillation]
                    x1 = tmp["x.2"]
                    y1 = tmp["y.2"]
                    x2 = tmp["x.3"]
                    y2 = tmp["y.3"]

                    (spb_x_pos, spb_y_pos) = middle_kinetochore_position(tmp["x"], tmp["y"], tmp["x.1"], tmp["y.1"])
                    (kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)
                    
                    
                    if kt_x_pos.any():
                        (x,time)=filter_signal(kt_x_pos,1,0.01)

                        # freq max = 1
                        (mag, freq) = get_fft(x, 0.05)
                        #amplitude=list()
                        #amplitude = list(mag)
                        #for elt in mag:
                        #    if elt > 0.01:
                        #        amplitude.append(elt)
                        amplitude = np.amax(mag)
                        d_centre.append(amplitude)
                        

                    solid.close()
                    ktmt1.close()
                    ktmt2.close()
        #if first==1:
        #    d={cohesinStiff:d_centre}
            #d_centre_tot = pd.DataFrame(data=d)
        #    first = 0
        
        #d={cohesinStiff:d_centre}
        #tmp = pd.DataFrame(data=d)
        #d_centre_tot.append(d)
        #d_centre_tot= pd.merge(d_centre_tot, tmp, how='right')
        d_centre_tot.append(list(d_centre))
    
    
    fig,ax = plt.subplots(figsize=(10,5))
    ax.violinplot(d_centre_tot, showmeans=True, showextrema=False)
    ax.set_xticks(pos)
    ax.set_xticklabels(label)
    ax.set_ylim([0,1])
    ax.set_xlabel('Seuil MT', fontsize=15)
    ax.set_ylabel('Amplitude(µm)', fontsize=15)
    
    
    plt.savefig("../Figures/MT_KTMT/figure_amplitude_KTMT/"+str(KTMT)+"KTMTStiffness.png")
    plt.close(fig)



In [ ]:
solid  = open("metaphase1kt_100MTseuil_100cohesinStiffness_200KTMTstiffness/run2/solid.txt")
ktmt1  = open("metaphase1kt_100MTseuil_100cohesinStiffness_200KTMTstiffness/run2/couple:KTMT.txt")
ktmt2  = open("metaphase1kt_100MTseuil_100cohesinStiffness_200KTMTstiffness/run2/couple:KTMT2.txt")

data_solid = pd.read_table(solid, sep='\t', header=0, index_col=False, lineterminator='\n')
data_ktmt1 = pd.read_table(ktmt1, sep='\t', header=0, index_col=False, lineterminator='\n')
data_ktmt2 = pd.read_table(ktmt2, sep='\t', header=0, index_col=False, lineterminator='\n')

debut_oscillation = start_oscillation(data_ktmt1,data_ktmt2)
fin_oscillation = end_oscillation(data_solid)

tmp = data_solid[data_solid["time"]>debut_oscillation]
tmp = tmp[tmp["time"]<fin_oscillation]
x1 = tmp["x.2"]
y1 = tmp["y.2"]
x2 = tmp["x.3"]
y2 = tmp["y.3"]

(kt_x_pos, kt_y_pos) = middle_kinetochore_position(x1, y1, x2, y2)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 10))

axes[0].violinplot(d_centre, showmeans=True, showextrema=False)
#axes[0].set_title('magnitude')

pos   = [1]
label = ['100']
axes[0].set_xticks(pos)
axes[0].set_xticklabels(label)
axes[0].set_xlabel('Stiffness', fontsize=15)
axes[0].set_ylabel('centre', fontsize=15)


axes[1].violinplot(d_stretch, showmeans=True, showextrema=False)
#axes[1].set_title('frequence')
axes[1].set_xticks(pos)
axes[1].set_xticklabels(label)
axes[1].set_xlabel('Stiffness', fontsize=15)
axes[1].set_ylabel('stretch', fontsize=15)

plt.show()
